In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
from tensorflow import keras
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers

from datagen import DataGenerator

In [2]:
batch_size=32
epochs=10
input_shape = (20,21,3)
classes = 4
train_gen = DataGenerator('dynamic_gestures2.hdf5')

In [3]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(32)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 63)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 64)            0     

In [4]:
model.fit_generator(generator=train_gen,epochs=epochs)

Epoch 1/10
C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
69/69 [==============================] - 4s 7ms/step - loss: 1.1842 - accuracy: 0.5305
Epoch 2/10
69/69 [==============================] - 0s 7ms/step - loss: 0.3060 - accuracy: 0.9104
Epoch 3/10
69/69 [==============================] - 0s 6ms/step - loss: 0.1169 - accuracy: 0.9719
Epoch 4/10
69/69 [==============================] - 0s 7ms/step - loss: 0.0688 - accuracy: 0.9802
Epoch 5/10
69/69 [==============================] - 0s 7ms/step - loss: 0.0544 - accuracy: 0.9841
Epoch 6/10
69/69 [==============================] - 0s 7ms/step - loss: 0.0392 - accuracy: 0.9891
Epoch 7/10
69/69 [==============================] - 0s 6ms/step - loss: 0.0260 - accuracy: 0.9947
Epoch 8/10
69/69 [=================

In [5]:
model.save("test_model_2", save_format = "tf")

INFO:tensorflow:Assets written to: test_model_2\assets
INFO:tensorflow:Assets written to: test_model_2\assets


In [4]:
!pip --version

pip 19.1.1 from C:\ProgramData\Anaconda3\lib\site-packages\pip (python 3.7)



In [3]:
x,y=[],[]
with h5py.File('dynamic_gestures2.hdf5', 'a') as f:
    print(f.filename)
    i=0
    for _,gesture in enumerate(list(f['/'])):
        if not "swipe" in gesture:
            continue
        print(i,gesture)
        data = list(f['/'+gesture])
        x.extend(data)
        y.extend([i]*len(data))
        i+=1
        if gesture == "swipe_right":
            print(i,"swipe_left")
        elif gesture == "swipe_up":
            print(i,"swipe_down")
        data = list(np.flip(np.array(list(f['/'+gesture])), axis=1))
        x.extend(data)
        y.extend([i]*len(data))
        i+=1

x=np.array(x)
y=np.array(y) 
y=tf.keras.utils.to_categorical(y,classes)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=True)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

dynamic_gestures2.hdf5
0 swipe_right
1 swipe_left
2 swipe_up
3 swipe_down
(669, 20, 21, 3) (75, 20, 21, 3) (669, 9) (75, 9)
